Testing Propotype with Selenium & google library
*** I have using FireFox for the selenium web driver. Be sure that Firefox has been installed on the local machine.

In [ ]:
pip install selenium
!pip install google

In [8]:
#from googlesearch import search_news
from googlesearch import search
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException

import pandas as pd
import time
import traceback


In [11]:
OUTPUT_FILE = 'output.csv'
AUTHOR_MAX_WORDS = 3

# Function for Getting Content from url
def getContent(driver,url):
    try:
        driver.get(url)
        H1_elements = driver.find_elements(By.TAG_NAME, "h1")
        p_elements = driver.find_elements(By.TAG_NAME, "p")
        authors = ''
        copyright = ''
        paragraph = ''
        title = ''
        # Getting Title
        for h1 in H1_elements:
            try:
                title = h1.text
                break  # Assuming you only need the first h1's text
            except Exception as e:
                print("Error accessing h1 text:", e)
                continue

        # Getting Content
        for p in p_elements:
            try:
                value = p.text
                # Filter out authors
                if value.startswith('By '):
                    authors = value.replace('By ', '') 
                    words = authors.split()
                    if len(words) <= AUTHOR_MAX_WORDS and not authors:
                        authors = value
                        continue
                # Filter out Copyright
                if '©' in value or 'All Rights Reserved' in value:
                    copyright = value
                    continue
                paragraph += value + ' '
            except Exception as e:
                print(f"Error accessing paragraph text from {url}:\n", e)
                continue
    except TimeoutException:
        print(f"Timeout waiting for elements on the page: {url}")
    except NoSuchElementException:
        print(f"An element was not found on the page: {url}")
    except Exception as e:
        print(f"An unexpected error occurred with URL => {url}: {e}")

    return {'authors': authors, 'title' : title, 'content': paragraph, 'url' : url, 'copyright': copyright }

def save_to_csv_with_pandas(recordSet, filename):
    # Convert the list of dictionaries to a DataFrame
    df = pd.DataFrame(recordSet)
    # Save the DataFrame to a CSV file
    df.to_csv(filename, index=False, encoding='utf-8')



In [97]:
# define query String
queryString = 'Did Joe Biden tell voters to stay home and not vote?'

#Search in google url with google library
results = search(query=queryString,num = 10 )

In [103]:
dataSet = []
driver = webdriver.Firefox()
i = 0
for url in results:
    #time.sleep(3)
    dataSet.append(getContent(driver,url))
driver.close()
df = pd.DataFrame(dataSet)
df

error found in url =>https://www.economist.com/briefing/2024/01/04/joe-bidens-chances-do-not-look-good-the-democrats-have-no-plan-b
error found in url =>https://www.aljazeera.com/news/2020/11/3/americans-choose-biden-or-trump-unprecedented-election-live-news


,authors,title,content,url,copyright
0,,Democrats Are Trailing Everywhere But at the B...,THE REPORT Democrats aren’t getting a lot...,https://www.usnews.com/news/the-report/article...,
1,,"Biden loses ground with young voters, promptin...",Democrats are raising concerns that President ...,https://thehill.com/homenews/campaign/4304919-...,© 1998 - 2024 NEXSTAR MEDIA INC. | ALL RIGHTS ...
2,,Saving democracy is central to Biden’s campaig...,Some Pennsylvania voters say President Biden’...,https://apnews.com/article/democracy-threats-b...,Copyright 2024 The Associated Press. All Right...
3,"2019, Thomas began singing a decidedly differe...",Biden’s Young Voter Problem Just Got Even Worse,Young voters are proving decidedly less intere...,https://newrepublic.com/post/177667/bidens-you...,
4,"late Thursday, three voting rights groups had ...",Biden slams GOP vote-restriction bills as ‘sic...,President Joe Biden on Thursday slammed a slew...,https://www.cnn.com/2021/03/25/politics/voting...,© 2024 Cable News Network. A Warner Bros. Disc...
...,...,...,...,...,...
94,,The 2020 Election by the Numbers,It’s almost over. Yesterday ...,https://www.cfr.org/blog/2020-election-numbers,
95,,,,https://www.youtube.com/watch?v=2HHqcr43qr0,
96,Alex Samuels,Biden Is Running For Reelection. What Are His ...,"2024 ELECTION APR. 25, 2023, AT 7:37 AM PHOTO ...",https://fivethirtyeight.com/features/biden-ree...,© 2024 ABC News Internet Ventures. All rights ...
97,,Can Joe Biden Intervene in a House Speaker Vot...,As the chaos that has embroiled the House Repu...,https://www.newsweek.com/can-joe-biden-interve...,


In [105]:
save_to_csv_with_pandas(df, OUTPUT_FILE)